In [2]:
import cv2
import os
import mediapipe as mp
from mediapipe.tasks import python
import numpy as np

path = './'  # 原图路径
# cropped_save_path = 'cropped_pic/' + img_path  # 眼部区域存储路径

model_path = 'face_landmarker.task'

# 加载mediapipe对象
BaseOptions = mp.tasks.BaseOptions
FaceLandmarker = mp.tasks.vision.FaceLandmarker
FaceLandmarkerOptions = mp.tasks.vision.FaceLandmarkerOptions
VisionRunningMode = mp.tasks.vision.RunningMode
options = FaceLandmarkerOptions(
    base_options=BaseOptions(model_asset_path=model_path),
    running_mode=VisionRunningMode.IMAGE)
landmarker = FaceLandmarker.create_from_options(options)

imgs_ = os.listdir(path)

imgs = []
for i in imgs_:
    if '2' in i:
        imgs.append(i)
        
# with open('list.txt', 'w') as file:
#     file.write(';'.join(str(element) for element in imgs))

for img_list in imgs:
    cropped_save_path = 'cropped_img/' + img_list
    imgs_list = os.listdir(img_list)
    for img in imgs_list:
        mp_image = mp.Image.create_from_file(img_list + '/' + img)
        face_landmarker_result = landmarker.detect(mp_image)
        mp_width = mp_image.width
        mp_height = mp_image.height
        image = cv2.imread(img_list + '/' + img)
        if not face_landmarker_result.face_landmarks:
            print(img_list + '/' + img)
        else:
        # 获取得到的特征点坐标
            landmarks = face_landmarker_result.face_landmarks[0]
            left_up_x = int(landmarks[53].x * mp_width)
            left_up_y = int(landmarks[53].y * mp_height)
            right_down_x = int(landmarks[346].x * mp_width)
            right_down_y = int(landmarks[346].y * mp_height)
            eye_region_width = right_down_x - left_up_x
            eye_region_height = int(0.375 * eye_region_width)
            fix_ycenter = int((right_down_y + left_up_y) * 0.5)
            #     print(right_down_y)
            #     print(left_up_y)
            #     print(fix_ycenter)
            fix_left_up_y = fix_ycenter - int(0.5 * eye_region_height)
            region_cropped_image = image[fix_left_up_y:fix_left_up_y + eye_region_height, left_up_x:left_up_x + eye_region_width]
            cv2.imwrite(os.path.join(cropped_save_path, img), region_cropped_image)

20210222linfan/07.JPG
